In [ ]:
#cargamos la data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score,confusion_matrix, roc_auc_score, average_precision_score, roc_curve, precision_recall_curve, auc


df=pd.read_csv('breast cancer.csv')
df.head()   

#preparamos los datos y eliminamos una columna con valores nulos
df.drop(['Unnamed: 32', 'id'], axis=1, inplace=True)

#imprimimos las nuevas columnas
variables=df.columns
print(variables)


#escogemos las variables y aplicamos el modelo de regresion logistica
X = df[['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean']]
y = df['diagnosis']

# Mapear las etiquetas 'M' (maligno) a 1 y 'B' (benigno) a 0
y = y.map({'M': 1, 'B': 0})

# Crea una instancia del modelo de regresión logística
model = LogisticRegression()

# Ajusta el modelo a los datos
model.fit(X, y)

# Realiza predicciones con el modelo
y_pred = model.predict(X)

# Calcula la precisión del modelo
accuracy = accuracy_score(y, y_pred)
print(f'Precisión del modelo: {accuracy}')

# Calcula y muestra la matriz de confusión
confusion = confusion_matrix(y, y_pred)
print('Matriz de Confusión:')
print(confusion)

# Genera un informe de clasificación que incluye precisión, recuperación y F1-score
report = classification_report(y, y_pred)
print('Informe de Clasificación:')
print(report)



#evaluamos el modelo, calculamos las metricas de evaluacion para el modelo
# Obtener las probabilidades de predicción
y_prob = model.predict_proba(X)[:, 1]

# Calcular el AUC-ROC
roc_auc = roc_auc_score(y, y_prob)
print(f'AUC-ROC: {roc_auc}')

# Calcular el AUC-PR
prc_auc = average_precision_score(y, y_prob)
print(f'AUC-PR: {prc_auc}')

# Calcular la curva ROC
fpr, tpr, thresholds_roc = roc_curve(y, y_prob)

# Calcular la curva PRC
precision, recall, thresholds_prc = precision_recall_curve(y, y_prob)

# Graficar la curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='Curva ROC (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos (FPR)')
plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
plt.title('Curva ROC')
plt.legend(loc='lower right')

# Graficar la curva PRC
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, color='blue', lw=2, label='Curva PRC (AUC = %0.2f)' % prc_auc)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Curva Precision-Recall')
plt.legend(loc='lower left')

# Mostrar las curvas ROC y PRC
plt.show()





#desbalanceo de clases luego veremos el efecto que provoca en el modelo de regresion logistica
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler,LabelEncoder

# Cargamos los datos
df=pd.read_csv('creditcard.csv')
# Eliminamos Duplicados
df.drop_duplicates(inplace=True)


# Eliminamos Nulos, y los datos "Time" y "Amount" por no estar correctamente codificadas
df=df.dropna()
df=df.drop(['Time','Amount'] , axis = 1)
col=df.columns.to_list()

# Es necesario escalar
sc=StandardScaler()

# Identifiquemos las variables X e y
X=df.drop('Class', axis = 1)
Y=df['Class']
X=sc.fit_transform(X)
print(df['Class'].value_counts())


#como podemos ver la clase 0 tiene muchos mas que la clase 1, usaremos SMOTE para corregirlo

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
from imblearn.over_sampling import SMOTE

X = df.drop(columns=["Class"]).values

# Variable dependiente
y = df["Class"].values

# Estandarizar las características
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Modelo de Regresión Logística sin SMOTE
logistic_regression_model = LogisticRegression(random_state=42)
logistic_regression_model.fit(X, y)

# Realizar predicciones en el conjunto de datos sin SMOTE
y_pred = logistic_regression_model.predict(X)

# Calcular AUC-ROC sin SMOTE
roc_auc = roc_auc_score(y, y_pred)

# Calcular AUC-PR sin SMOTE
average_precision = average_precision_score(y, y_pred)

# Ahora, aplicaremos SMOTE y crearemos el modelo nuevamente
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Modelo de Regresión Logística con SMOTE
logistic_regression_model_smote = LogisticRegression(random_state=42)
logistic_regression_model_smote.fit(X_resampled, y_resampled)

# Realizar predicciones en el conjunto de datos con SMOTE
y_pred_smote = logistic_regression_model_smote.predict(X)

# Calcular AUC-ROC con SMOTE
roc_auc_smote = roc_auc_score(y, y_pred_smote)

# Calcular AUC-PR con SMOTE
average_precision_smote = average_precision_score(y, y_pred_smote)

# Imprimir AUC-ROC y AUC-PR para ambos modelos
print("AUC-ROC sin SMOTE:", roc_auc)
print("AUC-PR sin SMOTE:", average_precision)
print("AUC-ROC con SMOTE:", roc_auc_smote)
print("AUC-PR con SMOTE:", average_precision_smote)

In [ ]:
Un aumento en el AUC-ROC con SMOTE es común, ya que SMOTE mejora la separación entre las clases, pero una disminución en el AUC-PR puede ocurrir debido a la introducción de ejemplos sintéticos que pueden no ser ideales para la clasificación precisa de la clase minoritaria. La elección de qué métrica es más importante depende del contexto de tu problema. Si la precisión en la detección de la clase minoritaria es crucial, entonces debes prestar más atención al AUC-PR y considerar ajustes adicionales en el modelo o en la técnica de muestreo.